<B>This NoteBook is for Segmenting and Clustering Neighborhoods in Toronto.</B></n></n>

Resolve dependancy if any, Install beautifulsoup4 and request pakage. 



In [1]:
!pip install beautifulsoup4
!pip install requests

distributed 1.21.8 requires msgpack, which is not installed.
distributed 1.21.8 requires msgpack, which is not installed.


<H3>Steps</H3>

i)   Import required libraries like pd, beautifulSoup, request.

ii)  Get the html request object for wikipedia web of postal codes of canada.

iii) Using beautifulsoup object get table tag data.

iv)  Iterate through HTML table, read all table row <tr>  and table data <td> and create list
    
V)   Create Pandas Dataframe from list.

In [2]:
import pandas as pd           # library to process data as dataframes
from bs4 import BeautifulSoup # library to process html page
import requests               # library to request data object

html = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(html, "lxml")

table = soup.find('table', attrs={'class':'wikitable sortable'}) # Get HTML table tag data
table_rows = table.find_all('tr')                  ## find all table row in table

res = []
for tr in table_rows:                        ## Iterate and create all list from table data.
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)

### Create DataFrame from HTML table list in above step and name columns.
df = pd.DataFrame(res, columns=["PostalCode", "Borough", "Neighbourhood"])
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


<H3>Count the size of dataframe after creating from HTML table</H3>

In [3]:
df.shape

(289, 3)

<H3>Steps</H3>

i) Filter rows if any Borough have 'Not assigned' value.

ii) Replace Neighbourhood values with Borough if Neighbourhood is having value 'Not assigned'.

In [4]:
df = df.loc[df['Borough'] != 'Not assigned'].reset_index()  ## Ignore 'Not assigned' Borough's

df.Neighbourhood.replace('Not assigned',df.Borough,inplace=True) ## Repalce 'Not assigned' neighbourhood with Borough value

df.head(10)

,index,PostalCode,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,Harbourfront
3,5,M5A,Downtown Toronto,Regent Park
4,6,M6A,North York,Lawrence Heights
5,7,M6A,North York,Lawrence Manor
6,8,M7A,Queen's Park,Queen's Park
7,10,M9A,Etobicoke,Islington Avenue
8,11,M1B,Scarborough,Rouge
9,12,M1B,Scarborough,Malvern


<H3>Count the size of dataframe after filtering 'Not assigned' Borough</H3>

In [5]:
df.shape

(212, 4)

<H3>Steps</H3>

i) Where more than one neighborhood can exist in one postal code combine two rows into one row with the neighborhoods separated with a comma.

In [6]:
df = df.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(list).to_frame().reset_index() 

df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]
5,M1J,Scarborough,[Scarborough Village]
6,M1K,Scarborough,"[East Birchmount Park, Ionview, Kennedy Park]"
7,M1L,Scarborough,"[Clairlea, Golden Mile, Oakridge]"
8,M1M,Scarborough,"[Cliffcrest, Cliffside, Scarborough Village West]"
9,M1N,Scarborough,"[Birch Cliff, Cliffside West]"


<H3>Show the shape of the final data frame</H3>

In [7]:
df.shape

(103, 3)